In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = labeling(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

In [ ]:
output_df = pd.DataFrame(data = None, columns = sample_submission.columns)
count = 0
minsize = 25
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n],cutoff=Step_level))
    for i in range(0,len(rle)):
        if len(rle[i]) <= minsize: 
            continue
        else:
            cell_annotations = ' '.join([str(x) for x in rle[i]])
            output_df.loc[count] = id_,cell_annotations
            count +=1

In [ ]:
output_df.to_csv('submission.csv', index = False)
pd.read_csv('submission.csv').head()

In [ ]:
transfile = pd.read_csv('submission.csv')

In [ ]:
i = 2
path = TEST_PATH + test_ids[i]
img = cv2.imread(path + '.png')
imr = test_ids[i]
_,axs = plt.subplots(1,2,figsize=(40,15))
axs[0].imshow(img)
for ids,row in transfile.iterrows():
    if row.id ==imr:
        rle = transfile.loc[row.name, 'predicted']
        dec = rle_decode(rle,shape=(520, 704))
        axs[1].imshow(np.ma.masked_where(dec==0, dec))